<a href="https://colab.research.google.com/github/Dylan-Geraci/ParkSafe-LA/blob/main/notebooks/modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [5]:
file_path_final_df = '/content/drive/My Drive/machine-learning/park-safe-la/data/Grouped_df.csv'
df = pd.read_csv(file_path_final_df)
df.head()

,day_of_week,ticket_count,hour_sin,hour_cos,zip_90001,zip_90002,zip_90003,zip_90004,zip_90005,zip_90006,zip_90007,zip_90008,zip_90009,zip_90010,zip_90011,zip_90012,zip_90013,zip_90014,zip_90015,zip_90016,zip_90017,zip_90018,zip_90019,zip_90020,zip_90021,zip_90022,zip_90023,zip_90024,zip_90025,zip_90026,zip_90027,zip_90028,zip_90029,zip_90031,zip_90032,zip_90033,zip_90034,zip_90035,zip_90036,zip_90037,...,zip_91396,zip_91401,zip_91402,zip_91403,zip_91405,zip_91406,zip_91409,zip_91411,zip_91412,zip_91413,zip_91416,zip_91423,zip_91436,zip_91501,zip_91502,zip_91504,zip_91505,zip_91506,zip_91508,zip_91601,zip_91602,zip_91603,zip_91604,zip_91605,zip_91606,zip_91607,zip_91608,zip_91614,zip_91617,zip_91711,zip_91748,zip_91765,zip_91767,zip_91768,zip_91769,zip_91776,zip_91793,zip_91803,zip_other,risk_level
0,Monday,1,9.659258e-01,0.258819,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,low
1,Friday,1,1.224647e-16,-1.000000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,low
2,Tuesday,2,7.071068e-01,-0.707107,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,low
3,Wednesday,1,5.000000e-01,-0.866025,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,low
4,Wednesday,1,-8.660254e-01,-0.500000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,low


In [9]:
le = LabelEncoder()
df['risk_label'] = le.fit_transform(df['risk_level'])

le.classes_

array(['high', 'low', 'medium'], dtype=object)

In [12]:
X = df.drop(columns=['risk_level', 'risk_label', 'ticket_count'])
y = df['risk_label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [13]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'Sunday'